In [2]:
# this is the dataset Bianca sent me
# I called the fodler dataset_dirty
# and created a folder called dataset where I will put the images and annotations
# after I clean them up
images_path = 'dataset_dirty/images/Alreadydone'
annotations_path = 'dataset_dirty/annoations'

In [8]:
# images and annotations have the same name but different extensions, we need to match them
import os

# Get list of images and annotations
images = [f for f in os.listdir(images_path) if f.endswith('.png')]
annotations = [f for f in os.listdir(annotations_path) if f.endswith('.json')]

# Create dictionaries with base filenames as keys
images_dict = {os.path.splitext(f)[0]: f for f in images}
annotations_dict = {os.path.splitext(f)[0]: f for f in annotations}

# Find matches
matches = {k: (images_dict[k], annotations_dict[k]) for k in images_dict if k in annotations_dict}

print('Matches:', len(matches))

Matches: 165


In [14]:
# now that they are matched, we are going to create the actual dataset folder
# the dataset folder will be divided by train, valid and test (70%, 20%, 10%)
# inside if each of those folders, there is a folder called images and another called annotations
# we will copy the images and annotations to the corresponding folder
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the directory for the dataset
dataset_dir = 'dataset'

# Create the main directories
os.makedirs(os.path.join(dataset_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'train', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'valid', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'valid', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'test', 'annotations'), exist_ok=True)

# Get a list of all matched keys
matched_keys = list(matches.keys())

# Split the keys into train, valid, and test
train_keys, test_keys = train_test_split(matched_keys, test_size=0.1, random_state=42)
train_keys, valid_keys = train_test_split(train_keys, test_size=0.22, random_state=42)  # 0.22 x 0.9 = 0.2

# Function to copy files
def copy_files(keys, split):
    for key in keys:
        image_file, annotation_file = matches[key]
        shutil.copy(os.path.join(images_path, image_file), os.path.join(dataset_dir, split, 'images', image_file))
        shutil.copy(os.path.join(annotations_path, annotation_file), os.path.join(dataset_dir, split, 'annotations', annotation_file))

# Copy files
copy_files(train_keys, 'train')
copy_files(valid_keys, 'valid')
copy_files(test_keys, 'test')

In [15]:
# now we go inside the dataset folder to train/annotations, valid/annotations and test/annotations and we are going to change the json files
# on the json files we remove 'imagePath' and 'imageData'

import json
import glob

# Define the splits and the keys to remove
splits = ['train', 'valid', 'test']
keys_to_remove = ['imagePath', 'imageData']

# Function to modify JSON files
def modify_json_files(directory):
    # Get a list of all JSON files in the directory
    json_files = glob.glob(os.path.join(directory, '*.json'))

    for json_file in json_files:
        with open(json_file, 'r') as f:
            data = json.load(f)

        # Remove the keys
        for key in keys_to_remove:
            data.pop(key, None)

        # Write the modified data back to the file
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)

# Modify JSON files in each split
for split in splits:
    modify_json_files(os.path.join(dataset_dir, split, 'annotations'))

Now that the data is in the disired format, we can start developing our code to actually train the model
You can check that on notebook.ipynb